In [1]:
import requests
import json

import pandas as pd
from pandas.tseries.frequencies import to_offset

import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import math

from sklearn.preprocessing import MinMaxScaler

import time
import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.metrics import mean_absolute_percentage_error

import warnings
warnings.filterwarnings('ignore')

In [2]:
segment_size = 100

# 1727 rows
data = pd.read_csv('./Dataset/HAG.csv')

# Calculate the number of segments in the dataset
print("len(data)", len(data))

data['ma_5'] = data['close_price'].ewm(span=3, adjust=False, min_periods=0).mean()

# Xóa bỏ các dòng có giá trị null do moving average
data.dropna(inplace=True)


len(data) 1720


In [3]:
# closedf = data[['trunc_time','ma_5']]
closedf = data[['trunc_time','close_price','ma_5']]
print(closedf)
print("Shape of close dataframe:", closedf.shape)

      trunc_time  close_price          ma_5
0     2016-01-07        10100  10100.000000
1     2016-01-08        10200  10150.000000
2     2016-01-11        10900  10525.000000
3     2016-01-12        11300  10912.500000
4     2016-01-13        10900  10906.250000
...          ...          ...           ...
1715  2022-11-21         7750   7329.238613
1716  2022-11-22         7800   7564.619307
1717  2022-11-23         7350   7457.309653
1718  2022-11-24         7860   7658.654827
1719  2022-11-25         8410   8034.327413

[1720 rows x 3 columns]
Shape of close dataframe: (1720, 3)


In [4]:
import copy

close_stock = closedf.copy()
del closedf['trunc_time']

X_data = copy.deepcopy(closedf)
Y_data = copy.deepcopy(closedf)

del X_data['close_price']
del Y_data['ma_5']

scaler=MinMaxScaler(feature_range=(0,1))

X_data=scaler.fit_transform(np.array(X_data).reshape(-1,1))
Y_data=scaler.fit_transform(np.array(Y_data).reshape(-1,1))

print(X_data, X_data.shape)
print(Y_data, Y_data.shape)

[[0.59684186]
 [0.60081252]
 [0.63059241]
 ...
 [0.38697773]
 [0.40296717]
 [0.43280048]] (1720, 1)
[[0.57633588]
 [0.58396947]
 [0.63740458]
 ...
 [0.36641221]
 [0.40534351]
 [0.44732824]] (1720, 1)


In [5]:
del closedf['close_price']
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))

print(closedf.shape)

(1720, 1)


In [6]:
training_size=int(len(closedf)*0.8)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)
# train_data

train_data:  (1376, 1)
test_data:  (344, 1)


In [7]:

def create_dataset(data_set):
    training_size=int(len(data_set)*0.8)
    train_data,test_data=data_set[0:training_size,:],data_set[training_size:len(data_set),:1]
    print("train_data: ", train_data.shape)
    print("test_data: ", test_data.shape)
    return train_data,test_data

In [8]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 10

X_train, X_test = create_dataset(X_data)
y_train, y_test = create_dataset(Y_data)
print("X_train: ", X_train)
print("y_train: ", y_train)
print("X_test: ", X_test)
print("y_test", y_test)


train_data:  (1376, 1)
test_data:  (344, 1)
train_data:  (1376, 1)
test_data:  (344, 1)
X_train:  [[0.59684186]
 [0.60081252]
 [0.63059241]
 ...
 [0.23206454]
 [0.21989121]
 [0.21936345]]
y_train:  [[0.57633588]
 [0.58396947]
 [0.63740458]
 ...
 [0.21221374]
 [0.20229008]
 [0.2129771 ]]
X_test:  [[0.21433479]
 [0.19752611]
 [0.19865133]
 [0.19921395]
 [0.19989232]
 [0.19943737]
 [0.19206273]
 [0.19036073]
 [0.18911267]
 [0.19245929]
 [0.18817662]
 [0.1812705 ]
 [0.18655288]
 [0.18919407]
 [0.19210292]
 [0.19236615]
 [0.19368896]
 [0.1963357 ]
 [0.20004146]
 [0.21420336]
 [0.22088725]
 [0.22224386]
 [0.2225251 ]
 [0.22147453]
 [0.21658152]
 [0.2121497 ]
 [0.20834552]
 [0.21081115]
 [0.21124983]
 [0.20591026]
 [0.20324048]
 [0.19753787]
 [0.19468656]
 [0.19326091]
 [0.19334221]
 [0.19457406]
 [0.19518998]
 [0.19748327]
 [0.19982111]
 [0.20297536]
 [0.20137596]
 [0.19819387]
 [0.19779402]
 [0.19402051]
 [0.19689853]
 [0.19476396]
 [0.19369667]
 [0.19276596]
 [0.19825659]
 [0.20298723]
 [0

In [9]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_depth=26, min_samples_leaf=2, min_samples_split=10,n_estimators=160, max_features=1)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=26, max_features=1, min_samples_leaf=2,
                      min_samples_split=10, n_estimators=160)

In [10]:
# Lets Do the prediction 

RF_train_predict=rf.predict(X_train)
RF_test_predict=rf.predict(X_test)

RF_train_predict = RF_train_predict.reshape(-1,1)
RF_test_predict = RF_test_predict.reshape(-1,1)

print("Train data prediction:", RF_train_predict.shape)
print("Test data prediction:", RF_test_predict.shape)

Train data prediction: (1376, 1)
Test data prediction: (344, 1)


In [11]:
# Transform back to original form

RF_train_predict = scaler.inverse_transform(RF_train_predict)
RF_test_predict = scaler.inverse_transform(RF_test_predict)
RF_original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
RF_original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [12]:
# Evaluation metrices RMSE and MAE
RF_RMSE_train = math.sqrt(mean_squared_error(RF_original_ytrain,RF_train_predict))
RF_MSE_train = mean_squared_error(RF_original_ytrain,RF_train_predict)
RF_MAE_train = mean_absolute_error(RF_original_ytrain,RF_train_predict)

RF_RMSE_test = math.sqrt(mean_squared_error(RF_original_ytest,RF_test_predict))
RF_MSE_test = mean_squared_error(RF_original_ytest,RF_test_predict)
RF_MAE_test = mean_absolute_error(RF_original_ytest,RF_test_predict)

print("Train data RMSE: ", RF_RMSE_train)
print("Train data MSE: ", RF_MSE_train)
print("Test data MAE: ", RF_MAE_train)
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", RF_RMSE_test)
print("Test data MSE: ", RF_MSE_test)
print("Test data MAE: ", RF_MAE_test)

Train data RMSE:  85.10464229861145
Train data MSE:  7242.800140774606
Test data MAE:  57.186664093706895
-------------------------------------------------------------------------------------
Test data RMSE:  1460.8821339063848
Test data MSE:  2134176.609166872
Test data MAE:  910.0892229486784


In [13]:
RF_EV_train = explained_variance_score(RF_original_ytrain, RF_train_predict)
RF_EV_test = explained_variance_score(RF_original_ytest, RF_test_predict)

print("Train data explained variance regression score:", RF_EV_train)
print("Test data explained variance regression score:", RF_EV_test)

Train data explained variance regression score: 0.9971371485927742
Test data explained variance regression score: 0.8392823499649856


In [14]:
RF_r2_train = r2_score(RF_original_ytrain, RF_train_predict)
RF_r2_test = r2_score(RF_original_ytest, RF_test_predict)

print("Train data R2 score:", RF_r2_train)
print("Test data R2 score:", RF_r2_test)

Train data R2 score: 0.9971371440258691
Test data R2 score: 0.7629382806978422


In [15]:
RF_MGD_train = mean_gamma_deviance(RF_original_ytrain, RF_train_predict)
RF_MGD_test = mean_gamma_deviance(RF_original_ytest, RF_test_predict)
RF_MPD_train = mean_poisson_deviance(RF_original_ytrain, RF_train_predict)
RF_MPD_test = mean_poisson_deviance(RF_original_ytest, RF_test_predict)
print("Train data MGD: ", RF_MGD_train)
print("Test data MGD: ", RF_MGD_test)
print("----------------------------------------------------------------------")
print("Train data MPD: ", RF_MPD_train)
print("Test data MPD: ",RF_MPD_test)

Train data MGD:  0.00018103552237850958
Test data MGD:  0.01740332992710613
----------------------------------------------------------------------
Train data MPD:  1.0960823205414483
Test data MPD:  192.06008735984668


In [16]:
# shift train predictions for plotting
from itertools import cycle
import plotly.express as px

print(RF_train_predict.shape)
print(RF_test_predict.shape)
print(closedf.shape)
print(len(close_stock))

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[0:len(RF_train_predict), :] = RF_train_predict

print("Train predicted data: ", trainPredictPlot)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(RF_train_predict):len(closedf), :] = RF_test_predict

print("Test predicted data: ", testPredictPlot.shape)

close_stock['Predictions']=testPredictPlot
close_stock['Stock']='HAG'
close_stock['Model']='SkLearn'
close_stock['Method']='MV'
close_stock.to_csv('./output/HAG_SkLearn_MV.csv', index=False)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['trunc_time'],
                       'original_close': close_stock['ma_5'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

(1376, 1)
(344, 1)
(1720, 1)
1720
Train predicted data:  [[ 9967.83933364]
 [ 9967.83933364]
 [10161.69872858]
 ...
 [           nan]
 [           nan]
 [           nan]]
Test predicted data:  (1720, 1)
